See: https://medium.com/@akelleh/causal-inference-with-pandas-dataframes-fc3e64fce5d and https://github.com/akelleh/causality

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

plt.style.use("ggplot")

In [2]:
from causality.inference.search import IC
from causality.inference.independence_tests import RobustRegressionTest


/Users/jattenberg/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
# generate some toy data:
SIZE = 2000
x1 = np.random.normal(size=SIZE)
x2 = x1 + np.random.normal(size=SIZE)
x3 = x1 + np.random.normal(size=SIZE)
x4 = x2 + x3 + np.random.normal(size=SIZE)
x5 = x4 + np.random.normal(size=SIZE)

# load the data into a dataframe:
X = pd.DataFrame({'x1' : x1, 'x2' : x2, 'x3' : x3, 'x4' : x4, 'x5' : x5})

# define the variable types: 'c' is 'continuous'.  The variables defined here
# are the ones the search is performed over  -- NOT all the variables defined
# in the data frame.
variable_types = {'x1' : 'c', 'x2' : 'c', 'x3' : 'c', 'x4' : 'c', 'x5' : 'c'}

# run the search
ic_algorithm = IC(RobustRegressionTest)
graph = ic_algorithm.search(X, variable_types)
graph.edges(data=True)

[('x2', 'x1', {'arrows': [], 'marked': False}),
 ('x2', 'x4', {'arrows': ['x4'], 'marked': False}),
 ('x3', 'x1', {'arrows': ['x3'], 'marked': False}),
 ('x3', 'x4', {'arrows': ['x4', 'x4'], 'marked': True}),
 ('x3', 'x5', {'arrows': ['x3', 'x5'], 'marked': False}),
 ('x4', 'x5', {'arrows': ['x5'], 'marked': True})]

In [4]:
from causality.estimation.adjustments import AdjustForDirectCauses
from networkx import DiGraph

g = DiGraph()

g.add_nodes_from(['x1','x2','x3','x4', 'x5'])
g.add_edges_from([('x1','x2'),('x1','x3'),('x2','x4'),('x3','x4')])
adjustment = AdjustForDirectCauses()
print adjustment.admissable_set(g,['x2'], ['x3'])

set(['x1'])


In [5]:
from causality.estimation.nonparametric import CausalEffect
admissable_list = list(adjustment.admissable_set(g,['x2'], ['x3']))
effect = CausalEffect(X, ['x2'], ['x3'], variable_types=variable_types, admissable_set=admissable_list)

In [6]:
x = pd.DataFrame({'x2' : [0.], 'x3' : [0.]})
effect.pdf(x)

0.2780128871117751